In [1]:
import pandas as pd
import matplotlib as mt

from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np

import re
from itertools import groupby

import MDAnalysis as mda
from MDAnalysis.analysis import align, rms
from MDAnalysis.tests.datafiles import PSF, DCD, PDB_small

from statsmodels.nonparametric.smoothers_lowess import lowess
import progressbar

In [2]:
table=pd.read_excel('Pep4/Results/Interactions_Table.xlsx',index_col=[0])
table.head()

,Time,Type,Protein,Peptide,AccType,DonType,WaterIdx,DistanceAWat,DistanceDWat,AngleDon,...,PosAtoms,PosAtomsIdx,ProtIsPos,RecRingType,LigRingType,RecRingAtoms,RecAtomsIdx,LigRingAtoms,LigRingAtomsIdx,Offset
Frame,,,,,,,,,,,,,,,,,,,,,
0,0,waterbridge,TYR473_R,GLU3_P,O3,O.co2,844.0,4.062278,2.598884,100.127071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,hbond,ASN487_R,ASN4_P,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,waterbridge,LYS417_R,ARG6_P,Ng+,N3,801.0,2.696090,3.243224,137.087178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,waterbridge,LYS417_R,ARG6_P,N3,Ng+,808.0,2.895963,3.821806,121.030379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,waterbridge,TYR421_R,ARG6_P,O2,Ng+,808.0,2.845505,3.821806,121.030379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
interaction_types=list(set(table['Type']))
print (interaction_types)

['saltbridge', 'hbond', 'hydroph_interaction', 'pistack', 'waterbridge']


time=table['Time'].drop_duplicates()
index=table.index.drop_duplicates()
hbond=[]
saltbridge=[]
pication=[]
hydroph=[]
pistack=[]
wb=[]
for x in index: 
    hbond.append(len([i for i in table.loc[x,'Type'] if i=='hbond']))
    wb.append(len([i for i in table.loc[x,'Type'] if i=='waterbridge']))
    saltbridge.append(len([i for i in table.loc[x,'Type'] if i=='saltbridge']))
    pication.append(len([i for i in table.loc[x,'Type'] if i=='pication']))
    hydroph.append(len([i for i in table.loc[x,'Type'] if i=='hydroph_interaction']))
    pistack.append(len([i for i in table.loc[x,'Type'] if i=='pistack']))

plt.rcParams['axes.linewidth'] = 1.5

fig, ax = plt.subplots(figsize=(10,5))

ax.scatter([i/100 for i in index],[float('nan') if x==0 else x for x in hbond],c='k',s=0.5, marker='.')
#ax.scatter([i/1000 for i in time],wb,c='C4',s=1, marker='x',label='Water Bridge')

plt.title ('Pep1',fontsize=24,fontweight='bold',color='k')
plt.xlabel ('Time (ns)',fontsize=20,fontweight='bold')
plt.ylabel ('Frequency\n(π-Cation)',fontsize=20,fontweight='bold')

plt.tick_params ('both',width=2,labelsize=14)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

#plt.tight_layout()

plt.savefig('Pep1_Hbond.png',dpi=300,format='png',bbox_inches='tight')
plt.show()

table=table[table['Type']!='hydroph_interaction']

In [ ]:
interaction_types=list(set(table['Type']))
heatmap=pd.DataFrame(index=sorted(list(set(table['Protein']))), columns=sorted(set(table['Peptide'])))

for i in list(set(table['Protein'])):
    residue=sorted(list(table[table['Protein']==i]['Peptide']))
    groups=groupby(residue)
    for x in groups:
        heatmap.loc[i,x[0]]=len(list(x[1]))

In [ ]:
heatmap.columns=[i.split('_')[0] for i in heatmap.columns]
heatmap.index=[i.split('_')[0] for i in heatmap.index]

cols=list(heatmap.columns)
cols.sort(key=lambda res: int(re.split('(\d+)',res)[1]))
ndx=list(heatmap.index)
ndx.sort(key=lambda res: int(re.split('(\d+)',res)[1]))
heatmap = heatmap[cols]
heatmap=heatmap.reindex(ndx)

In [ ]:
heatmap=heatmap.transpose().fillna(0)
heatmap = heatmap[(heatmap.T != 0).any()]

In [ ]:
heatmap.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8,3))

sns.heatmap (heatmap,yticklabels=heatmap.index, xticklabels=heatmap.columns,vmax=6500,cmap='Reds',cbar_kws=dict(label='Frequency',shrink=1,orientation='vertical',spacing='uniform',pad=0.02))

plt.title('Peptide 6',size='18',weight='bold',color='hotpink')
plt.ylabel('Peptide residue',fontsize=14,fontweight='bold')
plt.xlabel('RBD residue',fontsize=14,fontweight='bold')
plt.xticks (rotation=90,fontsize=5)
plt.yticks (fontsize=8)

#ax.xaxis.tick_top()
plt.tick_params ('both',width=1.5)
plt.savefig('Pep6/Results/Interactions_HM.png',dpi=300,format='png',bbox_inches='tight')
plt.show()

In [4]:
inter=[]
for i in table.index:
    try:
        inter.append([i,len(table.loc[i,'Time'])])
    except:
        pass

In [5]:
inter.sort(key=lambda x: x[1])

In [6]:
inter[-1]

[2, 30]

In [7]:
u=mda.Universe ('Pep4/Results/equilibration.gro','Pep4/Results/production_fit.xtc')
u

<Universe with 49135 atoms>

In [8]:
for res in u.residues:
    if res.resname=='TIP3':
        res.resname='HOH'
    if 'HI' in res.resname or 'HSD' in res.resname:
        res.resname='HIS'
    if 'CY' in res.resname:
        res.resname='CYS'
for atom in u.atoms:
    if atom.name=='OH2':
        atom.name='OW'

pep_segment = u.add_Segment(segid='P')
pep = u.select_atoms('resid 1:23')
pep.residues.segments=pep_segment

rec_segment = u.add_Segment(segid='R')
rec=u.select_atoms('protein and (not segid P)')
rec.residues.segments=rec_segment

In [9]:
with mda.Writer("Pep4/Results/Frame.pdb", u) as PDB:
    for ts in u.trajectory[2:3]:
        PDB.write(u.atoms)

/mnt/c/Users/angel/Linux_programs/miniconda3/envs/AnalysisMD/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  "".format(attrname, default))
/mnt/c/Users/angel/Linux_programs/miniconda3/envs/AnalysisMD/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  "".format(attrname, default))
/mnt/c/Users/angel/Linux_programs/miniconda3/envs/AnalysisMD/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  "".format(attrname, default))
/mnt/c/Users/angel/Linux_programs/miniconda3/envs/AnalysisMD/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'tempfactors' Using default value of '0.0'
  "".format(attrname, default))


# MD analysis

## RMSD heatmap

In [ ]:
u=mda.Universe ('Spike_RBD/Results/equilibration.gro','Spike_RBD/Results/production_fit.xtc')
u

In [ ]:
pep_segment = u.add_Segment(segid='P')
pep = u.select_atoms('resid 1:23')
pep.residues.segments=pep_segment

rec_segment = u.add_Segment(segid='R')
rec=u.select_atoms('protein and (not segid P)')
rec.residues.segments=rec_segment

In [ ]:
Receptor=u.select_atoms('segid R and (resid 333:526)')
print (Receptor)

Peptide=u.select_atoms('segid P')
print (Peptide)

In [ ]:
bar=progressbar.ProgressBar(max_value=len(u.trajectory[:5000:100]))
RMSD_hmap=pd.DataFrame()
for i in range(len(u.trajectory[:5000:100])):
    for j in range(len(u.trajectory[:5000:100])):
        bb = Receptor.select_atoms('backbone')
        u.trajectory[i]
        A = bb.positions.copy() # coordinates of first frame
        u.trajectory[j]         # forward to last frame
        B = bb.positions.copy()  # coordinates of last frame
        rmsd=rms.rmsd(A, B, center=True)
        RMSD_hmap.loc[i,j]=rmsd
    bar.update(i)

In [ ]:
RMSD_hmap.to_excel('Spike_RBD/Results/RMSD_Hmap.xlsx')

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

n = 10
while len(RMSD_hmap)/n > 6:
    n += 10
    
ax=sns.heatmap(RMSD_hmap,square=True,xticklabels=n,yticklabels=n,vmin=0, vmax=2.2,cmap='RdBu_r',cbar_kws=dict(label='RMSD (Å)',shrink=1,orientation='vertical',spacing='uniform',pad=0.02))

plt.title('apo-RBD',size=26,weight='bold',color='k')
plt.ylabel('Time (ns)',fontsize=22,fontweight='bold', rotation=90)
plt.xlabel('Time (ns)',fontsize=22,fontweight='bold', rotation=0)

#ax.xaxis.tick_top()
plt.tick_params ('both',width=2,labelsize=18)
cax = plt.gcf().axes[-1]
cax.tick_params(labelsize=16)
ax.figure.axes[-1].yaxis.label.set_size(22)
plt.tight_layout()
plt.savefig('Spike_RBD/Results/RMSD_HM_RBD.png',quality=95,dpi=300,format='png',bbox_inches='tight')
plt.show()

## RMSF

In [ ]:
system=u.select_atoms('segid R and (resid 333:526)')
calphas = system.select_atoms("name CA")
rmsfer = rms.RMSF(calphas, verbose=True).run(stop=5000)

In [ ]:
plt.rcParams['axes.linewidth'] = 1.5
plt.figure(figsize=(8,5))
plt.plot(calphas.resnums, rmsfer.rmsf,linewidth=1.5,color='k')
plt.xlabel (' Residue αC',fontsize=16,fontweight='bold')
plt.ylabel ('RMSF (Å)',fontsize=16,fontweight='bold')
plt.tick_params ('both',width=2,labelsize=12)
plt.grid (axis='y')
plt.tight_layout()
plt.savefig('Spike_RBD/Results/RBD_RMSF.png',dpi=600,format='png',transparent=False)
plt.show()

In [ ]:
save=pd.DataFrame(rmsfer.rmsf,index=calphas.resnums)
save.to_csv('Spike_RBD/Results/RBD_RMSF.csv')

In [ ]:
u.add_TopologyAttr('tempfactors')

In [ ]:
rmsf=[]
for atom in system.atoms:
    rmsf.append(save.loc[atom.resid,0])

In [ ]:
with mda.Writer("Spike_RBD/Results/RBD.pdb", system.n_atoms) as PDB:
    for ts in u.trajectory[0:1]:
        system.atoms.tempfactors = rmsf
        PDB.write(system.atoms)

In [ ]:
array=[[min(save[0]), max(save[0])],[min(save[0]) , max(save[0])]]
plt.imshow(array,cmap='jet')
m=plt.colorbar(orientation='vertical',aspect=10)
m.set_label ('RMSF (Å)',fontsize=20,fontweight='bold')

m.ax.tick_params(labelsize=16) 

plt.savefig('Spike_RBD/Results/Bfactor_bar.png',quality=95,dpi=600,format='png',transparent=False)

In [ ]:
system=u.select_atoms('protein')
with mda.Writer("Spike_RBD/Results/RBD_snapshots.pdb", system.n_atoms) as PDB:
    for ts in u.trajectory[0:5000:1000]:
        PDB.write(system.atoms)